In [1]:
############ADD FEATURES AND RESPONSE############

In [2]:
# import required modules
import sqlite3
import pandas as pd
import numpy as np
import ta

In [3]:
def df_from_db(dbname):
    """
    Setup sqlite connection and returns the dataframe
    """
    conn = sqlite3.connect(dbname)
    # load table to dataframe
    tdf = pd.read_sql("SELECT * FROM eow", 
                 conn, parse_dates="date")
    conn.close()
    print(tdf.head())
    return tdf

In [4]:
def optimize_columns(tdf):
    """
    Optimize the memory allocation of the columns
    """
    tdf['open'] = tdf['open'].astype(float)
    tdf['high'] = tdf['high'].astype(float)
    tdf['low'] = tdf['low'].astype(float)
    tdf['close'] = tdf['close'].astype(float)
    tdf['adjclose'] = tdf['adjclose'].astype(float)
    tdf['volume'] = tdf['volume'].astype(int)
    print(tdf.memory_usage())
    return tdf

In [5]:
def index_operations(tdf):
    """
    Performs preliminery index operations
    """
    # Make symbol as index
    tdf.set_index(keys= "symbol", inplace = True, append=False)
    # Make date as index
    tdf.set_index(keys= "date", inplace = True, append=True)
    # Sort the entire dataset
    tdf.sort_index(inplace=True)
    print(tdf.head())
    return tdf

In [6]:
def add_response(tdf):
    """
    Add response variables:
    shifted logret and shifted adjclose
    """
    # Add logret
    tdf['logret'] = np.log(tdf.adjclose).groupby(tdf.company).diff()
    # Shift the log returns
    tdf['slogret'] = tdf['logret'].shift(-1)
    # Shifted adjclose
    tdf['sadjclose'] = tdf.adjclose.groupby(tdf.company).shift(-1)
    # Drop last row of each ticker, (has NAN)
    tdf = tdf.dropna(axis=0)
    print(tdf.head())
    return tdf

In [7]:
def print_stats(df):
    print("########COLUMN NAN##############")
    print("MAX:",df.isnull().sum(axis = 1).groupby(df.company).sum().min())
    print("MIN:",df.isnull().sum(axis = 1).groupby(df.company).sum().max())
    print(df.isnull().sum(axis = 1).groupby(df.company).sum())
    print("########ROW NAN##############")
    print("MAX:", df.isnull().sum(axis = 0).max())
    print("MIN:",df.isnull().sum(axis = 0).min())
    print(df.isnull().sum(axis = 0))
    print("########DF SHAPE##############")
    print(df.shape)
    print("########DF HEAD###############")
    print(df.head)
    return

In [8]:
# Store it in the database
def df_to_db(tdf, dbname):
    """
    save database as sqlite db
    """
    conn = sqlite3.connect(dbname)
    tdf.to_sql(name='eow', con=conn, if_exists='replace', index=True, index_label=['symbol', 'date'])
    conn.close()
    return

In [ ]:
############################################

In [9]:
df = df_from_db("stockseow.db")
df = optimize_columns(df)
df = index_operations(df)

  symbol       date company     open     high    low  close  adjclose  \
0      A 2011-01-07       A  41.5600  42.1391  41.00  41.62   27.3601   
1      A 2011-01-14       A  41.4100  43.4100  41.30  43.26   28.4382   
2      A 2011-01-21       A  43.3700  44.4500  41.46  42.11   27.6822   
3      A 2011-01-28       A  42.1834  43.5200  40.88  40.98   26.9394   
4      A 2011-02-04       A  41.2100  43.1300  40.23  42.99   28.2607   

     volume  
0  23040100  
1  17917600  
2  21525200  
3  22294000  
4  27637800  
Index           128
symbol      1679200
date        1679200
company     1679200
open        1679200
high        1679200
low         1679200
close       1679200
adjclose    1679200
volume      1679200
dtype: int64
                  company     open     high    low  close  adjclose    volume
symbol date                                                                  
A      2011-01-07       A  41.5600  42.1391  41.00  41.62   27.3601  23040100
       2011-01-14       A  41.

In [10]:
## remove LHX, just 145 values
grouped = df.groupby('company')
df = df.drop(grouped.get_group('LHX').index)

In [11]:
# Check if all companies have equal entries
grouped = df.adjclose.groupby(df.company)
grouped.count().min() == grouped.count().max()

True

In [12]:
df.head()

company     open     high    low  close  adjclose    volume
symbol date                                                                  
A      2011-01-07       A  41.5600  42.1391  41.00  41.62   27.3601  23040100
       2011-01-14       A  41.4100  43.4100  41.30  43.26   28.4382  17917600
       2011-01-21       A  43.3700  44.4500  41.46  42.11   27.6822  21525200
       2011-01-28       A  42.1834  43.5200  40.88  40.98   26.9394  22294000
       2011-02-04       A  41.2100  43.1300  40.23  42.99   28.2607  27637800

In [13]:
# Add all features
ta.add_all_ta_features(
    df, open="open", high="high", low="low", close="close", volume="volume", fillna=True)
df.head()

/home/ya5ha5/anaconda3/envs/rnpython/lib/python3.7/site-packages/ta/trend.py:468: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
/home/ya5ha5/anaconda3/envs/rnpython/lib/python3.7/site-packages/ta/trend.py:472: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


company     open     high    low  close  adjclose    volume  \
symbol date                                                                     
A      2011-01-07       A  41.5600  42.1391  41.00  41.62   27.3601  23040100   
       2011-01-14       A  41.4100  43.4100  41.30  43.26   28.4382  17917600   
       2011-01-21       A  43.3700  44.4500  41.46  42.11   27.6822  21525200   
       2011-01-28       A  42.1834  43.5200  40.88  40.98   26.9394  22294000   
       2011-02-04       A  41.2100  43.1300  40.23  42.99   28.2607  27637800   

                     volume_adi  volume_obv  volume_cmf  ...  momentum_uo  \
symbol date                                              ...                
A      2011-01-07  2.040862e+06    23040100    0.088579  ...     1.340688   
       2011-01-14  1.741094e+07    40957700    0.425096  ...     5.335547   
       2011-01-21  5.244519e+06    19432500    0.083935  ...     6.290787   
       2011-01-28 -1.536054e+07    -2861500   -0.181188  ...     6.168388   
       2011-02-04  9.608781e+06    24776300    0.085476  ...    10.705823   

                   momentum_stoch  momentum_stoch_signal  momentum_wr  \
symbol date                                                             
A      2011-01-07       54.428935              54.428935   -45.571065   
       2011-01-14       93.775934              74.102434    -6.224066   
       2011-01-21       32.173913              60.126261   -67.826087   
       2011-01-28        2.801120              42.916989   -97.198880   
       2011-02-04       65.402844              33.459292   -34.597156   

                   momentum_ao  momentum_kama  momentum_roc  others_dr  \
symbol date                                                              
A      2011-01-07          0.0          41.62           0.0 -52.295222   
       2011-01-14          0.0          43.26           0.0   3.940413   
       2011-01-21          0.0          42.11           0.0  -2.658345   
       2011-01-28          0.0          40.98           0.0  -2.683448   
       2011-02-04          0.0          42.99           0.0   4.904832   

                   others_dlr  others_cr  
symbol date                               
A      2011-01-07    0.000000   0.000000  
       2011-01-14    3.864760   3.940413  
       2011-01-21   -2.694318   1.177319  
       2011-01-28   -2.720110  -1.537722  
       2011-02-04    4.788339   3.291687  

[5 rows x 68 columns]

In [14]:
df = add_response(df)

                  company     open   high    low  close  adjclose    volume  \
symbol date                                                                   
A      2011-01-14       A  41.4100  43.41  41.30  43.26   28.4382  17917600   
       2011-01-21       A  43.3700  44.45  41.46  42.11   27.6822  21525200   
       2011-01-28       A  42.1834  43.52  40.88  40.98   26.9394  22294000   
       2011-02-04       A  41.2100  43.13  40.23  42.99   28.2607  27637800   
       2011-02-11       A  43.4500  45.15  42.57  45.02   29.5952  27674300   

                     volume_adi  volume_obv  volume_cmf  ...  momentum_wr  \
symbol date                                              ...                
A      2011-01-14  1.741094e+07    40957700    0.425096  ...    -6.224066   
       2011-01-21  5.244519e+06    19432500    0.083935  ...   -67.826087   
       2011-01-28 -1.536054e+07    -2861500   -0.181188  ...   -97.198880   
       2011-02-04  9.608781e+06    24776300    0.085476  ... 

In [15]:
print_stats(df)

########COLUMN NAN##############
MAX: 0
MIN: 0
company
A       0
AAL     0
AAP     0
AAPL    0
ABC     0
       ..
XRAY    0
XRX     0
YUM     0
ZBH     0
ZION    0
Length: 461, dtype: int64
########ROW NAN##############
MAX: 0
MIN: 0
company       0
open          0
high          0
low           0
close         0
             ..
others_dlr    0
others_cr     0
logret        0
slogret       0
sadjclose     0
Length: 71, dtype: int64
########DF SHAPE##############
(208833, 71)
########DF HEAD###############
<bound method NDFrame.head of                   company     open     high    low  close  adjclose    volume  \
symbol date                                                                     
A      2011-01-14       A  41.4100  43.4100  41.30  43.26   28.4382  17917600   
       2011-01-21       A  43.3700  44.4500  41.46  42.11   27.6822  21525200   
       2011-01-28       A  42.1834  43.5200  40.88  40.98   26.9394  22294000   
       2011-02-04       A  41.2100  43.1300  40.23  42

In [16]:
df_to_db(df, 'stocks-eow.db')

In [ ]:
########################################################